In [57]:
#TRYING TO MAKE BIDIRECTIONAL
#ISSUE WITH HIDDEN SIZE SHAPE NOT SURE ABOUT THIS ADDITION STUFF
#MAYBE FIND CODE FRO A BIDIRECTIONAL LSTM AS INSPO
#GOTO START HERE

In [1]:
from dataset_class import SleepDataset, SleepChunkDataset

In [24]:
import codecs
import math
import random
import string
import time
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score, recall_score, f1_score
import torch.nn as nn
from torch.autograd import Variable
from torchmetrics.classification import MulticlassCohenKappa
import torch
import torch.optim as optim
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from dataset_class import SleepDataset, SleepChunkDataset
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

### ActiNet

In [3]:

class DeepACTINeT(nn.Module):
    def __init__(self):
        super(DeepACTINeT, self).__init__()
        # Define the layers as per the structure provided
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=512, stride=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=256, stride=2)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=256, stride=2)
        self.conv4 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=32, stride=2)
        
        self.dropout = nn.Dropout(0.25)
        self.batch_norm1 = nn.BatchNorm1d(16)
        self.batch_norm2 = nn.BatchNorm1d(16)
        self.batch_norm3 = nn.BatchNorm1d(16)
        self.batch_norm4 = nn.BatchNorm1d(16)
        
        # self.fc1 = nn.Linear(16, 64)  # Adjust input size based on final pooled size
        # self.fc2 = nn.Linear(64, 10)  # Replace 10 with the number of output classes
    
    def forward(self, x):
        assert not torch.isnan(x).any(), "NaN detected in CNN input"
        x = x.permute(0,2, 1) #flip channels and num samples [batch, ch, num samples]

        initial_samples = x.shape[2]
        #print(initial_samples)
        #you want to end up with one every 5 seconds 
        final_samples = initial_samples // (32*5)
        print(final_samples)

        
        print(f'start shape {x.shape}')
        x = F.relu(self.conv1(x))
        print(f"Input shape after conv1: {x.shape}")
        x = self.dropout(x)
        print(f"Input shape after, dropout: {x.shape}")
        x = self.batch_norm1(x)
        print(f"Input shape after conv1,dropout,batchnorm: {x.shape}")
        x = F.max_pool1d(x, kernel_size=2)
        print(f"Input shape after maxpool: {x.shape}")
        
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = self.batch_norm2(x)
        print(f"Input shape after conv2: {x.shape}")
        
        x = F.relu(self.conv3(x))
        x = self.dropout(x)
        x = self.batch_norm3(x)
        print(f"Input shape after conv3: {x.shape}")
        
        x = F.max_pool1d(x, kernel_size=2)
        print(f"Input shape after maxpool: {x.shape}")
        
        x = F.relu(self.conv4(x))
        x = self.dropout(x)
        x = self.batch_norm4(x)
        print(f"before avg pooling shape: {x.shape}")

        x = F.adaptive_avg_pool1d(x, final_samples)
        print(f"Final output shape: {x.shape}")
        
        return x


### LSTM

In [4]:
data_dir = '/scratch/npr264/BioDeepL/dreamt/physionet.org/files/dreamt/2.0.0/data_64Hz'

In [25]:
class SleepStager(pl.LightningModule):
    def __init__(self,
                cnn_output_channels: int = 16,
                pre_lstm_embedding_size: int = 16,
                lstm_hidden_size:   int = 128,
                lstm_layers:        int = 6,
                hidden_size2:       int = 64,
                num_sleep_stages:   int = 5,
                lr:                  float = 1e-3,
                weight_tensor:      torch.Tensor = None,
                debug:            bool = False):
        super().__init__()
        # self.save_hyperparameters()
        
        self.acc_cnn = DeepACTINeT()
        #add non linearity to the bvp and temp before lstm 
        self.embeddingLayer = nn.Linear(2, pre_lstm_embedding_size)
    
        self.lstm_input_size = pre_lstm_embedding_size + cnn_output_channels
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm = nn.LSTM(input_size=self.lstm_input_size,
                            hidden_size=lstm_hidden_size,
                            bias = True,
                            batch_first=True,
                            bidirectional = True)
        #From paper 
        self.hidden2 = nn.Linear(lstm_hidden_size*2, hidden_size2)
        self.classifier = nn.Linear(hidden_size2, num_sleep_stages)
    
        self.criterion = nn.CrossEntropyLoss(weight=weight_tensor, ignore_index=-1)
        self.lr        = lr
        self.debug      = debug
        self.kappa = MulticlassCohenKappa(num_classes=num_sleep_stages)


        self.dropout = nn.Dropout(p=0.5)
        
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        #self.outputLayer = nn.Linear(self.hidden_size*self.seq_len, self.output_size)

    def forward(self, acc, non_acc):
        """
        non_acc: (batch, non_acc_length, non_acc_dim)
        acc:     (batch, acc_length)  or  (batch, acc_length, 1)
        returns:
          y_hat (output_length, batch, num_sleep_stages)
        """
        # ensure ACC has a channel dim
        if acc.dim() == 2:
            acc = acc.unsqueeze(-1)          # now (batch, T, 1)
        
        # 1) ACC → CNN → (batch, num_cnn_feats, cnn_output_length)
        acc_feats = self.acc_cnn(acc)
        
        if self.debug:
            print(f"[DEBUG] ACC CNN output shape: {acc_feats.shape}") # (batch, num_cnn_feats, cnn_output_length)
            print(f"[DEBUG] NON_ACC input shape: {non_acc.shape}") # (batch, non_acc_length, non_acc_dim)

        #embed the non_acc data 
        emb_non_acc = self.embeddingLayer(non_acc)
        emb_non_acc = self.dropout(emb_non_acc)
        # 3) build LSTM input: (T', batch, feature_dim)
        a = acc_feats.permute(2, 0, 1)        # (lstm_seq_len, batch, cnn_output_features)
        b = emb_non_acc.permute(1, 0, 2)       # (lstm_seq_len, batch, non_acc_dim)
        if self.debug:
            print(f"[DEBUG] ACC CNN reshaped shape: {a.shape}") # (batch, num_cnn_feats, cnn_output_length)
            print(f"[DEBUG] NON_ACC reshaped shape: {b.shape}") # (batch, non_acc_length, non_acc_dim)


        batch_size = acc.shape[0]
        hidden = model.init_hidden(batch_size).to(device)
        cell = model.init_hidden(batch_size).to(device)

        lstm_in = torch.cat([a, b], dim = 2)    # (lstm_seq_len, batch, C_cnn + D_nonacc)
        # without batch_first = true, LSTM input shape is (seq_len, batch_size, features)

#             [DEBUG] ACC CNN output shape: torch.Size([8, 16, 120])
# [DEBUG] NON_ACC input shape: torch.Size([8, 120, 2])
# [DEBUG] LSTM input shape: torch.Size([120, 8, 18])

        if self.debug:
            print(f"[DEBUG] LSTM input shape: {lstm_in.shape}")
    
        # 4) LSTM + classifier
        output,(hidden, cell) = self.lstm(lstm_in, (hidden, cell))
        if self.debug:
            print(f"[DEBUG] LSTM output shape: {output.shape}")

        
        pre_output = self.hidden2(output)
        if self.debug:
            print(f"[DEBUG] hidden2 shape: {pre_output.shape}")
        y_hat = self.classifier(pre_output)     # (lstm_seq_len, batch, num_sleep_stages)
        if self.debug:
            print(f"[DEBUG] Classifier output shape: {y_hat.shape}")
        return y_hat
    def init_hidden(self, batch_size):
        return Variable(torch.randn(1,batch_size,self.hidden_size))
    
    def training_step(self, batch, batch_idx):
        acc, non_acc, labels = batch
        '''
        non_acc: (batch, non_acc_length, non_acc_dim)
        acc:     (batch, acc_length)  or  (batch, acc_length, 1)
        labels:  (batch, non_acc_length)
        '''
        y_hat = self(non_acc, acc)            # (lstm_seq_len, batch, num_sleep_stages)
        y_hat = y_hat.permute(1, 0, 2)        # (batch, lstm_seq_len, num_sleep_stages)
    
    
        # flatten
        batch_size, output_length, num_sleep_stages = y_hat.shape
        y_hat_flat = y_hat.reshape(batch_size * output_length, num_sleep_stages)
        labels_flat  = labels.reshape(batch_size * output_length)
        
        # calculate loss
        loss = self.criterion(y_hat_flat, labels_flat)
    
        self.log('train_loss', loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        acc, non_acc, labels = batch
        '''
        non_acc: (batch, non_acc_length, non_acc_dim)
        acc:     (batch, acc_length)  or  (batch, acc_length, 1)
        labels:  (batch, non_acc_length)
        '''
        y_hat = self(non_acc, acc)            # (lstm_seq_len, batch, num_sleep_stages)
        y_hat = y_hat.permute(1, 0, 2)        # (batch, lstm_seq_len, num_sleep_stages)
    
        # flatten
        batch_size, output_length, num_sleep_stages = y_hat.shape
        y_hat_flat = y_hat.reshape(batch_size * output_length, num_sleep_stages)
        labels_flat  = labels.reshape(batch_size * output_length)
        
        # calculate loss
        loss = self.criterion(y_hat_flat, labels_flat)
    
        # calculate accuracy
        predictions = torch.argmax(y_hat_flat, dim=1)
        mask = labels_flat != -1
        masked_preds = predictions[mask]
        masked_labels = labels_flat[mask]
        if masked_labels.numel() > 0:
            acc = (masked_preds == masked_labels).float().mean().item()
        else:
            acc = 0.0
    
        # calculate cohen's kappa
        mask = labels_flat != -1
        y_valid = labels_flat[mask]
        preds_valid = predictions[mask]
        if y_valid.numel() > 0:
            ckappa = self.kappa(preds_valid, y_valid)
        else:
            ckappa = torch.tensor(0.0, device=self.device)
    
        # log metrics
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc",  acc,  prog_bar=True)
        self.log("val_cohen_kappa", ckappa, prog_bar=True)
    
        self.kappa.reset()
    
        return {"val_loss": loss, "val_acc": acc}
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [8]:
train_dataset = SleepChunkDataset(subjects_list=["S003"],
                                 data_dir=data_dir,
                                 debug=False)
train_loader = DataLoader(train_dataset, batch_size=8, num_workers=1, shuffle=True)
#just a quick lil pass to see if all the shapes are lining up 
criterion = nn.CrossEntropyLoss()

In [21]:
#THIS IS WHAT UR TRYING TO RUN

model = SleepStager(
    cnn_output_channels=16,
    lstm_hidden_size=64,
    lstm_layers=2,
    num_sleep_stages=5,
    lr=1e-3,
    debug=True
)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()


SleepStager(
  (acc_cnn): DeepACTINeT(
    (conv1): Conv1d(3, 16, kernel_size=(512,), stride=(2,))
    (conv2): Conv1d(16, 16, kernel_size=(256,), stride=(2,))
    (conv3): Conv1d(16, 16, kernel_size=(256,), stride=(2,))
    (conv4): Conv1d(16, 16, kernel_size=(32,), stride=(2,))
    (dropout): Dropout(p=0.25, inplace=False)
    (batch_norm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (embeddingLayer): Linear(in_features=2, out_features=16, bias=True)
  (lstm): LSTM(32, 64, batch_first=True, bidirectional=True)
  (hidden2): Linear(in_features=128, out_features=64, bias=True)
  (classifier): Linear(in_features=64, out_features=5, bias=True)
  (criterio

In [22]:
for batch_idx, (data, targets) in enumerate(train_loader):
    optimizer.zero_grad()
    print(targets[0].shape)
    outputs = model(data['acc_data'], data['tempbvp_data'])
        # outputs: [120, 8, 5] → [8, 120, 5]
    outputs = outputs.permute(1, 0, 2)
    
    # Flatten: [8, 120, 5] → [960, 5]
    outputs = outputs.reshape(-1, outputs.shape[-1])
    
    # targets: [8, 120] → [960]
    targets = targets.reshape(-1)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    break



torch.Size([120])
120
start shape torch.Size([8, 3, 19200])
Input shape after conv1: torch.Size([8, 16, 9345])
Input shape after, dropout: torch.Size([8, 16, 9345])
Input shape after conv1,dropout,batchnorm: torch.Size([8, 16, 9345])
Input shape after maxpool: torch.Size([8, 16, 4672])
Input shape after conv2: torch.Size([8, 16, 2209])
Input shape after conv3: torch.Size([8, 16, 977])
Input shape after maxpool: torch.Size([8, 16, 488])
before avg pooling shape: torch.Size([8, 16, 229])
Final output shape: torch.Size([8, 16, 120])
[DEBUG] ACC CNN output shape: torch.Size([8, 16, 120])
[DEBUG] NON_ACC input shape: torch.Size([8, 120, 2])
[DEBUG] ACC CNN reshaped shape: torch.Size([120, 8, 16])
[DEBUG] NON_ACC reshaped shape: torch.Size([120, 8, 16])
[DEBUG] LSTM input shape: torch.Size([120, 8, 32])
[DEBUG] LSTM output shape: torch.Size([120, 8, 128])
[DEBUG] hidden2 shape: torch.Size([120, 8, 64])
[DEBUG] Classifier output shape: torch.Size([120, 8, 5])


In [24]:
outputs.shape

torch.Size([120, 5])

### REference code

In [5]:
class CharGRULSTM(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, output_size, seq_len, rnn_type = "lstm", bidirectional=True):
        super(CharGRULSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.hidden_size2 = hidden_size2
        self.output_size = output_size
        self.seq_len=seq_len
        self.rnntype = rnn_type
        self.embeddingLayer = nn.Linear(self.input_size, self.hidden_size)
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1
        if rnn_type == "gru":
            self.rnn = nn.GRU(
                self.hidden_size,
                self.hidden_size,
                batch_first=True,
                bidirectional=self.bidirectional
            )
        else:
            self.rnn = nn.LSTM(
                self.hidden_size,
                self.hidden_size,
                bias=True,
                batch_first=True,
                bidirectional=self.bidirectional
            )
            #self.rnn = nn.LSTMCell(self.hidden_size, self.hidden_size, bias=True)
        self.dropout = nn.Dropout(p=0.5)
        self.hidden2 = nn.Linear(self.hidden_size * self.num_directions, self.hidden_size2)
        # Only take the output from the final timestep
        self.outputLayer = nn.Linear(self.hidden_size2, self.output_size)
        
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        #self.outputLayer = nn.Linear(self.hidden_size*self.seq_len, self.output_size)
        
        self.softmax = nn.LogSoftmax(dim=1)
        self.predloss = nn.CrossEntropyLoss()


    def forward(self, batch, hidden, cell=None):
        batchemb = self.embeddingLayer(batch)
        batchemb = self.dropout(batchemb)
        if self.rnntype == "lstm":
            #output,(hidden, cell) = self.rnn(batchemb.view(batchemb.shape[0],-1,self.hidden_size), (hidden, cell))
            output,(hidden, cell) = self.rnn(batchemb, (hidden, cell))
            #hidden, cell = self.rnn(batchemb, (hidden, cell))
        else:
            hidden = self.rnn(batchemb, hidden)
        final_hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)  # [batch_size, hidden*2]
        output = self.hidden2(final_hidden)
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        #output = self.outputLayer(self.dropout(output.view(-1,self.seq_len*self.hidden_size)))
        output=self.outputLayer(output)
        output = self.softmax(output)
        if self.rnntype == "lstm":
            return output, hidden, cell
        else:
            return output, hidden


    def init_hidden(self, batch_size):
        #return Variable(torch.randn(1, self.hidden_size))
        num_dir = self.num_directions
        return Variable(torch.randn(num_dir,batch_size,self.hidden_size))

In [3]:
def normalize(data):
    names1=['left_temp',
       'right_temp',  'from_19_min', 
       'log_left_ENMO', 'log_right_ENMO',
       'log_left_ENMO_var', 'log_right_ENMO_var',
       'log_left_anglex_var', 'log_right_anglex_var', 
       'log_left_angley_var', 'log_right_angley_var',
       'log_left_anglez_var', 'log_right_anglez_var', 
       'left_anglex', 'left_angley', 'left_anglez',
       'right_anglex', 'right_angley', 'right_anglez',
       "log_left_ENMO_range","log_right_ENMO_range",
       "log_left_anglex_range","log_right_anglex_range",
       "log_left_angley_range","log_right_angley_range",
       "log_left_anglez_range","log_right_anglez_range"]
    names2=['participant_id','Disorder', 'sex', 'age','y']
    nums=[1,2,14,17,21,23,27,28,29,31,32,34,35,38,39,42,45,48,49,50,51,52,53,56,57,59,60]
    data_normalized=pd.DataFrame()
    others=pd.DataFrame()
    for num in nums:
        tmp=data[names1][data["participant_id"]==num]
        tmp=(tmp-tmp.mean())/tmp.std()
        data_normalized=pd.concat([data_normalized, tmp],axis=0,ignore_index=True)
        others_tmp=data[names2][data["participant_id"]==num]
        others=pd.concat([others, others_tmp],axis=0,ignore_index=True)
    data_normalized=pd.concat([others,data_normalized],axis=1)
    return data_normalized

# get expanded input
def get_LSTM_input(data, nums, back=0, front=0,level=2):
    names=['left_temp','right_temp', 'Disorder', 'sex', 
        'age', 'from_19_min', 
       'log_left_ENMO_var', 'log_right_ENMO_var',
       'log_left_anglex_var', 'log_right_anglex_var', 
        'log_left_angley_var', 'log_right_angley_var',
       'log_left_anglez_var', 'log_right_anglez_var', 
       'left_anglex', 'left_angley', 'left_anglez',
       'right_anglex', 'right_angley', 'right_anglez',
       'log_left_ENMO', 'log_right_ENMO',"log_left_ENMO_range",
       "log_right_ENMO_range","log_left_anglex_range",
       "log_right_anglex_range","log_left_angley_range",
       "log_right_angley_range","log_left_anglez_range",
       "log_right_anglez_range"]
    #nums=[1,2,14,17,21,23,27,28,29,31,32,34,35,38,39,42,45,48,49,50,51,52,53,56,57,59,60] 
    X_list=[]
    y_list=[]
    for num in nums:
        #X
        sub=data[data["participant_id"]==num]
        X=sub[names].values.astype(float)
        X=np.reshape(X,(-1,6,len(names)))
        n=X.shape[0]
        # get original X
        exp_X=X[back:n-front,:,:]

        # Add back
        for i in range(1,back+1):
            back_tmp=X[back-i:n-front-i,:,:]
            exp_X=np.concatenate((back_tmp,exp_X),axis=1)
        #print(ret_X.shape)

        # Add Front
        for i in range(1,front+1):
            front_tmp=X[back+i:n-front+i,:,:]
            exp_X=np.concatenate((exp_X, front_tmp),axis=1)
        #print(ret_X.shape)
        X_list.append(exp_X)
        # Y
        if level==2:
            y=sub[["b_y"]].values.astype(float)
        elif level==3:
            y=sub[["t_y"]].values.astype(float)
        y=y[0::6]
        y=np.array([i[0] for i in y])
        y=y[back:n-front]
        y_list.append(y)
    
    ret_X=np.concatenate(X_list,axis=0)
    ret_y=np.concatenate(y_list,axis=0)
    return ret_X,ret_y


########### Functions and packages for LSTM ################
class Dataset(Dataset):
    def __init__(self, X, y):
        self.len = len(X)           
        if torch.cuda.is_available():
            self.x_data = torch.from_numpy(X).float().cuda()
            self.y_data = torch.from_numpy(y).long().cuda()
        else:
            self.x_data = torch.from_numpy(X).float()
            self.y_data = torch.from_numpy(y).long()

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

def calculateAccuracy(y_pred, y):
    #y_pred and y are lists
    tmp=[1 if y_pred[i]==y[i] else 0 for i in range(len(y))]
    return sum(tmp)/len(y)
"""
def predict(model, X):
    # X in shape #samples*length of seq*#features, all on device
    preds=torch.torch.FloatTensor([]).to(device)
    hidden = model.init_hidden(batch_size=X.shape[0]).to(device)
    cell = model.init_hidden(batch_size=X.shape[0]).to(device)
    pred_y, hidden,cell = model(X, hidden,cell)    
    preds=torch.cat((preds,pred_y),0)
    _, top_i =  preds.data.topk(1)
    return top_i
"""
def predict(model, val_loader):
    # X in shape #samples*length of seq*#features, all on device
    preds=torch.torch.FloatTensor([]).to(device)
    for i, (records, labels) in enumerate(val_loader): 
        hidden = model.init_hidden(batch_size=records.shape[0]).to(device)
        cell = model.init_hidden(batch_size=records.shape[0]).to(device)
        pred_y, hidden,cell = model(records, hidden,cell)    
        preds=torch.cat((preds,pred_y),0)
    _, top_i =  preds.data.topk(1)
    return top_i,preds.data

def train(model, train_loader, validation_loader, loss_function, optimizer,num_epochs):
    total_step = len(train_loader)
    acc_train=[]
    loss_train=[]
    acc_val=[]
    #X_train=train_loader.dataset.x_data.to(device)
    #y_train=train_loader.dataset.y_data.to(device)
    # This took too much RAM!
    #X_val=validation_loader.dataset.x_data.to(device)
    y_val=validation_loader.dataset.y_data.to(device)
    #Initial hidden
    for epoch in range(num_epochs):
        y_pred_train=torch.torch.FloatTensor([]).to(device)
        y_train=torch.torch.LongTensor([]).to(device)
        for i, (records, labels) in enumerate(train_loader): 
            optimizer.zero_grad()
            hidden = model.init_hidden(batch_size=records.shape[0]).to(device)
            cell = model.init_hidden(batch_size=records.shape[0]).to(device)
            # Move tensors to the configured device
            records = records.to(device)
            labels = labels.to(device)
            # Forward pass with data in 30s
            pred_y, hidden,cell = model(records, hidden,cell)            
            loss = loss_function(pred_y, labels)
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            y_pred_train=torch.cat((y_pred_train,pred_y),0)
            y_train=torch.cat((y_train,labels),0)
         
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        #y_pred_train=model(X_train) #Cost too much RAM!
        _, top_i_train =  y_pred_train.data.topk(1)
        acc_train_tmp=calculateAccuracy(top_i_train[:,0].tolist(), y_train.tolist())
        acc_train.append(acc_train_tmp)
        loss_epoch=loss_function(y_pred_train, y_train).item()
        loss_train.append(loss_epoch)
        #-----------
        top_i_val, _ =predict(model, validation_loader)
        #-----------
        acc_val_tmp=calculateAccuracy(top_i_val[:,0].tolist(), y_val.tolist())
        acc_val.append(acc_val_tmp)
        
    return model, np.array(acc_train),np.array(loss_train), np.array(acc_val)